# 🚗 Sprint Challenge 4 – Previsão de Acidentes com LSTMs
## Case Sompo: Antecipando Padrões de Risco em Rodovias Brasileiras

---

**Equipe Big 5**
- Lucca Phelipe Masini - RM 564121
- Luiz Henrique Poss - RM 562177
- Luis Fernando de Oliveira Salgado - RM 561401
- Igor Paixão Sarak - RM 563726
- Bernardo Braga Perobeli - RM 562468

---

## 🎯 Target Escolhido: Regressão - Proporção de Acidentes Severos

**Prevemos o valor contínuo** da proporção de acidentes severos (com mortos ou feridos graves) em cada semana por estado.

- **Range**: 0% a 100% de acidentes severos
- **Interpretação**: Quanto maior o valor, maior o risco da semana

### Justificativa

Optamos por **regressão** em vez de classificação porque:

1. ✅ **Mais adequado aos dados** - Features temporais não capturam fatores críticos para classificação (clima, eventos)
2. ✅ **Informação mais rica** - Valor contínuo (ex: 28% vs 32%) é mais útil que categoria binária
3. ✅ **Métricas claras** - MAE e RMSE mostram erro médio real em pontos percentuais
4. ✅ **Honestidade científica** - Mostra o que o modelo CONSEGUE fazer, não força classificação artificial

---

## 📚 Passo 1: Instalação e Importação de Bibliotecas

Primeiro, vamos instalar e importar todas as bibliotecas necessárias para o projeto.


In [ ]:
!pip install openpyxl --quiet
print("✅ Bibliotecas instaladas!")

---

## 📥 Passo 2: Carregamento dos Dados

Carregamos os dados diretamente do GitHub para garantir reprodutibilidade total.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("📚 Bibliotecas importadas!")

---

## 🎯 Passo 3: Pré-processamento e Criação da Variável Target

Criamos a variável binária `severo` que identifica acidentes com mortos ou feridos graves.


In [ ]:
print("📥 Baixando dataset da PRF do GitHub...")

github_raw_url = 'https://raw.githubusercontent.com/9luis7/lstm-acidentes-prf/main/dados/datatran2025.xlsx'
output_filename = 'dados_acidentes.xlsx'

try:
    urllib.request.urlretrieve(github_raw_url, output_filename)
    df = pd.read_excel(output_filename)
    print(f"✅ Dataset carregado: {len(df):,} acidentes")
    print("\n📊 Período:", df['data_inversa'].min(), "até", df['data_inversa'].max())
    print("📊 Estados:", df['uf'].nunique(), "UFs")
except Exception as e:
    print(f"❌ Erro: {e}")
    raise

---

## 🔄 Passo 4: Agregação Semanal

Transformamos acidentes individuais em séries temporais semanais por estado.


In [ ]:
print("🎯 Criando variável target 'severo'...")

df['horario'] = pd.to_datetime(df['horario'], format='%H:%M:%S', errors='coerce').dt.time
df['severo'] = ((df['mortos'] > 0) | (df['feridos_graves'] > 0)).astype(int)

colunas_relevantes = ['data_inversa', 'horario', 'uf', 'br', 'km', 'pessoas', 'veiculos', 'severo']
df_limpo = df[colunas_relevantes].copy()
df_limpo['horario'].fillna(pd.to_datetime('12:00:00').time(), inplace=True)

print("✅ Variável 'severo' criada!")
print("\n📊 Distribuição:")
print(df_limpo['severo'].value_counts(normalize=True))

---

## 🎨 Passo 5: Feature Engineering

Criamos 12 features enriquecidas: temporais, sazonalidade e histórico (lags).


In [ ]:
print("🔄 Agregando dados em séries temporais semanais...")

df_indexed = df_limpo.set_index('data_inversa')

weekly_df = df_indexed.groupby([pd.Grouper(freq='W'), 'uf']).agg(
    total_acidentes=('severo', 'count'),
    acidentes_severos=('severo', 'sum'),
    pessoas_total=('pessoas', 'sum'),
    veiculos_total=('veiculos', 'sum'),
    pessoas_media=('pessoas', 'mean'),
    veiculos_media=('veiculos', 'mean')
).reset_index()

weekly_df['prop_severos'] = np.where(
    weekly_df['total_acidentes'] > 0,
    weekly_df['acidentes_severos'] / weekly_df['total_acidentes'],
    0
)

print(f"✅ Dados agregados: {len(weekly_df):,} semanas × estados")

---

## 🔢 Passo 6: Criação de Sequências Temporais

Criamos janelas de 8 semanas para prever a semana seguinte. Normalizamos os dados com MinMaxScaler.


In [ ]:
print("🎨 Criando features temporais e de histórico...")

# Temporais
weekly_df['dia_semana'] = weekly_df['data_inversa'].dt.dayofweek
weekly_df['mes'] = weekly_df['data_inversa'].dt.month
weekly_df['fim_semana'] = weekly_df['dia_semana'].isin([5, 6]).astype(int)

# Sazonalidade
weekly_df['sazonalidade_sen'] = np.sin(2 * np.pi * weekly_df['data_inversa'].dt.dayofyear / 365)
weekly_df['sazonalidade_cos'] = np.cos(2 * np.pi * weekly_df['data_inversa'].dt.dayofyear / 365)

# Lags
for lag in [1, 2, 3]:
    weekly_df[f'prop_severos_lag{lag}'] = weekly_df.groupby('uf')['prop_severos'].shift(lag)

# Estatísticas
weekly_df['prop_severos_ma3'] = weekly_df.groupby('uf')['prop_severos'].rolling(3).mean().reset_index(0, drop=True)
weekly_df['prop_severos_tendencia'] = weekly_df.groupby('uf')['prop_severos'].diff()
weekly_df['prop_severos_volatilidade'] = weekly_df.groupby('uf')['prop_severos'].rolling(3).std().reset_index(0, drop=True)

print("✅ Features criadas!")
print(f"   Total: {len(weekly_df.columns)} colunas")

---

## 🎯 Passo 7: Preparação para Regressão

Preparamos os dados para prever valores contínuos da proporção de acidentes severos.


---

## 🏗️ Passo 8: Construção do Modelo LSTM para Regressão

**Arquitetura para Regressão:**
- LSTM (64 neurônios) - captura padrões temporais
- Dense (32 → 16 neurônios) - processamento não-linear
- Output (1 neurônio, linear) - valor contínuo
- Loss: MAE (Mean Absolute Error)
- Métricas: MAE e MSE


In [ ]:
from sklearn.preprocessing import MinMaxScaler

print("🔢 Preparando sequências para LSTM (REGRESSÃO)...")

features_colunas = [
    'prop_severos', 'pessoas_media', 'veiculos_media', 'fim_semana',
    'sazonalidade_sen', 'sazonalidade_cos',
    'prop_severos_lag1', 'prop_severos_lag2', 'prop_severos_lag3',
    'prop_severos_ma3', 'prop_severos_tendencia', 'prop_severos_volatilidade'
]

df_features = weekly_df.set_index('data_inversa').sort_index()
df_features = df_features[features_colunas].copy()
df_features = df_features.dropna()

# Target: proporção de acidentes severos (valores contínuos 0-1)
target_values = df_features['prop_severos'].values

print("\n📊 Estatísticas do target (proporção de acidentes severos):")
print(f"   Min: {target_values.min():.3f} ({target_values.min()*100:.1f}%)")
print(f"   Max: {target_values.max():.3f} ({target_values.max()*100:.1f}%)")
print(f"   Média: {target_values.mean():.3f} ({target_values.mean()*100:.1f}%)")
print(f"   Mediana: {np.median(target_values):.3f} ({np.median(target_values)*100:.1f}%)")
print(f"   Desvio padrão: {target_values.std():.3f}")

# Separar features de target
features_sem_target = [col for col in features_colunas if col != 'prop_severos']
target_col = 'prop_severos'

# Normalizar apenas as FEATURES (sem o target)
df_features_input = df_features[features_sem_target].copy()
scaler_features = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler_features.fit_transform(df_features_input.values)

# TARGET permanece em escala original (0-1 já é uma escala natural)
target_values = df_features[target_col].values

n_passos_para_tras = 8
n_features = len(features_sem_target)

X, y = [], []
for i in range(n_passos_para_tras, len(features_scaled)):
    X.append(features_scaled[i-n_passos_para_tras:i, :])
    y.append(target_values[i])

X, y = np.array(X), np.array(y)

print(f"\n✅ Sequências criadas para regressão!")
print(f"   Shape X: {X.shape}")
print(f"   Shape y: {y.shape}")
print(f"   Range de y: [{y.min():.3f}, {y.max():.3f}]")

In [ ]:
print("🎯 Preparando dados para REGRESSÃO...")

# Dividir dados temporalmente (85% treino, 15% validação)
split_index = int(len(X) * 0.85)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

print(f"\n📊 Divisão temporal:")
print(f"   Treino: {len(X_train)} sequências ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Validação: {len(X_val)} sequências ({len(X_val)/len(X)*100:.1f}%)")

print(f"\n📊 Estatísticas de y_train:")
print(f"   Min: {y_train.min():.3f} ({y_train.min()*100:.1f}%)")
print(f"   Max: {y_train.max():.3f} ({y_train.max()*100:.1f}%)")
print(f"   Média: {y_train.mean():.3f} ({y_train.mean()*100:.1f}%)")
print(f"   Desvio: {y_train.std():.3f}")

print(f"\n📊 Estatísticas de y_val:")
print(f"   Min: {y_val.min():.3f} ({y_val.min()*100:.1f}%)")
print(f"   Max: {y_val.max():.3f} ({y_val.max()*100:.1f}%)")
print(f"   Média: {y_val.mean():.3f} ({y_val.mean()*100:.1f}%)")
print(f"   Desvio: {y_val.std():.3f}")

print("\n✅ Dados preparados para regressão!")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

print("🏗️  Construindo modelo LSTM para REGRESSÃO...")

# MODELO LSTM PARA REGRESSÃO
model = Sequential([
    LSTM(units=64, return_sequences=False, input_shape=(n_passos_para_tras, n_features)),
    Dropout(0.3),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=16, activation='relu'),
    Dense(units=1, activation='linear')  # REGRESSÃO: 1 neurônio, ativação linear
])

# OPTIMIZER
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='mean_absolute_error',  # MAE: erro médio absoluto em pontos percentuais
    metrics=['mae', 'mse']
)

print("✅ Modelo construído para REGRESSÃO!")
print("   Arquitetura: LSTM 64 → Dense 32 → Dense 16 → Linear 1")
print("   Dropout: 0.3 e 0.2")
print("   Loss: MAE (Mean Absolute Error)")
print("   Métricas: MAE e MSE")
print("   Learning rate: 0.001")
model.summary()

# CALLBACKS
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=1)

print("\n✅ Callbacks configurados!")
print("   Early Stopping: patience=20 (monitor val_loss/MAE)")
print("   Reduce LR: patience=10")

---

## 🚀 Passo 9: Treinamento do Modelo

Treinamos o modelo com class weights para corrigir desbalanceamento. Monitoramos val_loss para evitar overfitting.


In [ ]:
print("\n" + "="*70)
print("🚀 TREINANDO MODELO LSTM DE REGRESSÃO")
print("="*70)
print("\n📊 Prevendo proporção contínua de acidentes severos")
print("⏱️  Aguarde 10-20 minutos...\n")

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

print("\n" + "="*70)
print("✅ TREINAMENTO CONCLUÍDO!")
print("="*70)

---

## 🔧 Abordagem de Modelagem: Regressão

### Por que Regressão?

Após testes com classificação (4 classes e binária), identificamos que:

❌ **Classificação não funcionou** - Modelo sempre previa uma única classe  
❌ **Features limitadas** - Dados temporais não capturam fatores críticos (clima, eventos)  
✅ **Regressão é mais adequada** - Valores contínuos aproveitam melhor informação disponível

### Vantagens da Abordagem

1. **Informação rica** - Prever 28% vs 32% é mais útil que "risco baixo" vs "alto"
2. **Métricas claras** - MAE mostra erro médio em pontos percentuais
3. **Honestidade científica** - Mostra capacidade real do modelo
4. **Aplicação prática** - Gestores podem definir próprios thresholds de alerta

### Arquitetura

- **Input**: Sequências de 8 semanas × 11 features
- **LSTM**: 64 neurônios (captura padrões temporais)
- **Dense**: 32 → 16 neurônios (processamento não-linear)
- **Output**: 1 neurônio linear (valor contínuo 0-1)
- **Loss**: MAE (Mean Absolute Error)

---



---

## 📊 Passo 10: Avaliação e Métricas

Avaliamos o modelo com MAE, RMSE e R². Comparamos com baseline (sempre prever a média).


---

## 📈 Passo 11: Visualizações

Geramos 6 gráficos: curvas de aprendizagem (MAE/MSE), scatter plot real vs previsto, série temporal, distribuição de erros e análise de resíduos.


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("📊 Avaliando modelo de REGRESSÃO...")

# Predições
y_pred = model.predict(X_val, verbose=0).flatten()

# Métricas
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print("\n" + "="*70)
print("🎯 RESULTADOS FINAIS - REGRESSÃO")
print("="*70)

print(f"\n📊 Métricas de Erro:")
print(f"   MAE (Mean Absolute Error): {mae:.4f} ({mae*100:.2f} pontos percentuais)")
print(f"   RMSE (Root Mean Squared Error): {rmse:.4f} ({rmse*100:.2f} pontos percentuais)")
print(f"   R² Score: {r2:.4f}")

# Baseline: sempre prever a média
baseline_pred = np.full_like(y_val, y_train.mean())
baseline_mae = mean_absolute_error(y_val, baseline_pred)

print(f"\n📊 Comparação com Baseline (sempre prever média):")
print(f"   Baseline MAE: {baseline_mae:.4f} ({baseline_mae*100:.2f}pp)")
print(f"   Nosso modelo MAE: {mae:.4f} ({mae*100:.2f}pp)")
print(f"   Melhoria: {((baseline_mae - mae)/baseline_mae*100):.1f}%")

# Estatísticas dos erros
errors = np.abs(y_val - y_pred)
print(f"\n📊 Distribuição dos Erros Absolutos:")
print(f"   Mínimo: {errors.min():.4f} ({errors.min()*100:.2f}pp)")
print(f"   Máximo: {errors.max():.4f} ({errors.max()*100:.2f}pp)")
print(f"   Mediana: {np.median(errors):.4f} ({np.median(errors)*100:.2f}pp)")
print(f"   75º percentil: {np.percentile(errors, 75):.4f} ({np.percentile(errors, 75)*100:.2f}pp)")

print("\n" + "="*70)

---

## 💾 Passo 12: Salvamento do Modelo

Salvamos o modelo treinado no formato `.keras` para uso futuro.


In [ ]:
plt.figure(figsize=(16, 12))

# 1. Loss (MAE)
plt.subplot(3, 2, 1)
plt.plot(history.history['loss'], label='Treino', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validação', color='red', linewidth=2)
plt.title('Curvas de Aprendizagem - MAE', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('MAE')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. MSE
plt.subplot(3, 2, 2)
plt.plot(history.history['mse'], label='Treino', color='blue', linewidth=2)
plt.plot(history.history['val_mse'], label='Validação', color='red', linewidth=2)
plt.title('Curvas de Aprendizagem - MSE', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Real vs Previsto (Scatter Plot)
plt.subplot(3, 2, 3)
plt.scatter(y_val, y_pred, alpha=0.5, s=30)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Linha Perfeita')
plt.title('Real vs Previsto', fontsize=14, fontweight='bold')
plt.xlabel('Valor Real')
plt.ylabel('Valor Previsto')
plt.legend()
plt.grid(True, alpha=0.3)

# 4. Série Temporal: Real vs Previsto
plt.subplot(3, 2, 4)
plt.plot(y_val, label='Real', linewidth=2, alpha=0.7)
plt.plot(y_pred, label='Previsto', linewidth=2, alpha=0.7, linestyle='--')
plt.title('Comparação Temporal', fontsize=14, fontweight='bold')
plt.xlabel('Amostras')
plt.ylabel('Proporção de Acidentes Severos')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Distribuição dos Erros
plt.subplot(3, 2, 5)
errors = y_val - y_pred
plt.hist(errors, bins=30, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Erro Zero')
plt.title('Distribuição dos Erros (Resíduos)', fontsize=14, fontweight='bold')
plt.xlabel('Erro (Real - Previsto)')
plt.ylabel('Frequência')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Resíduos vs Valores Previstos
plt.subplot(3, 2, 6)
plt.scatter(y_pred, errors, alpha=0.5, s=30)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.title('Resíduos vs Valores Previstos', fontsize=14, fontweight='bold')
plt.xlabel('Valor Previsto')
plt.ylabel('Resíduo (Real - Previsto)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Visualizações geradas!")

In [ ]:
model_filename = 'modelo_lstm_classificacao_risco.keras'
model.save(model_filename)
print(f"💾 Modelo salvo: '{model_filename}'")
print("\n✅ Projeto concluído!")